# Langchain and Anthropic

Claude is chat-based model, meaning it is trained on conversation data. However, it is a text based API, meaning it takes in single string. It expects this string to be in a particular format. This means that it is up the user to ensure that is the case. LangChain provides several utilities and helper functions to make sure prompts that you write - whether formatted as a string or as a list of messages - end up formatted correctly.

Because Claude is chat-based but accepts a string as input, it can be treated as either a LangChain `ChatModel` or `LLM`. This means there are two wrappers in LangChain - ChatAnthropic and Anthropic. It is generally recommended to use the ChatAnthropic wrapper, and format your prompts as ChatMessages (we will show examples of this below). This is because it keeps your prompt in a general format that you can easily then also use with other models (should you want to). However, if you want more fine-grained control over the prompt, you can use the Anthropic wrapper - we will show and example of this as well. The `Anthropic` wrapper however is **deprecated**, as all functionality can be achieved in a more generic way using `ChatAnthropic`.

Ref: <https://python.langchain.com/docs/integrations/platforms/anthropic>

## Best Practices

#### No System Messages

Anthropic models are not trained on the concept of a "system message". We have worked with the Anthropic team to handle them somewhat appropriately (a Human message with an admin tag) but this is largely a hack and it is recommended that you do not use system messages.

#### AI Messages Can Continue

A completion from Claude is a continuation of the last text in the string which allows you further control over Claude's output.

For example, putting words in Claude's mouth in a prompt like this:

>`\n\nHuman: Tell me a joke about bears\n\nAssistant: What do you call a bear with no teeth?`

This will return a completion like this `A gummy bear!` instead of a whole new assistant message with a different random bear joke.

In [1]:
# !pip install -Uq boto3 anthropic-bedrock langchain rich

In [2]:
%load_ext rich

In [3]:
from langchain.chat_models import ChatAnthropic
from langchain.llms.bedrock import Bedrock
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from rich import print
import boto3
from anthropic_bedrock import HUMAN_PROMPT, AI_PROMPT
from types import FunctionType
from utils import get_inference_parameters
import tools
from pathlib import Path
import re
import warnings

warnings.filterwarnings("ignore")

### Initialize Bedrock LLM

Here we utilize `langchain.llms.bedrock.Bedrock` class to initialize our LLM.

In [4]:
# get bedrock runtime client
client = boto3.client("bedrock-runtime", region_name="us-west-2")

bedrock_model_id = "anthropic.claude-v2"  # Bedrock model_id
model_kwargs = get_inference_parameters("anthropic")  # Model kwargs for Anthropic LLMs
# print(model_kwargs)

bedrock_model = Bedrock(
    client=client, model_id=bedrock_model_id, model_kwargs=model_kwargs
)  # Initalize LLM

#### Creating prompts with ChatPromptTemplate

When working with ChatModels, it is preferred that you design your prompts as `ChatPromptTemplates`.

Here is an example below of doing that:

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        ("human", "Hi! How are you?"),
        ("assistant", "I'm doing well. Thanks for asking."),
        ("human", "Tell me a joke about {topic}"),
    ]
)
print(prompt)

ChatPromptTemplate(
    input_variables=['topic'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')
        ),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hi! How are you?')),
        AIMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template="I'm doing well. Thanks for asking.")
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['topic'], template='Tell me a joke about {topic}')
        )
    ]
)

You can then invoke the chain like below:

*NOTE:* The below example is using Langchain Expression Language (LCEL)

Refer to <https://python.langchain.com/docs/expression_language> for more details.

In [6]:
chain = prompt | bedrock_model
chain.invoke({"topic": "cats"})

" Here's a silly joke about cats:\n\nWhy don't cats play poker in the jungle? Too many cheetahs!"

### Passing inputs using HumanMessage

Below we use `HumanMessage` from `langchain.schema` to construct `messages` for the chat model.

We then create a `ChatPromptTemplate` from these messages to invoke this chain.


*NOTE: As there are no input variables to be passed, we send an empty `input` string*

In [7]:
from langchain.schema import HumanMessage

messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
prompt = ChatPromptTemplate.from_messages(messages=messages)

chain = prompt | bedrock_model
chain.invoke({"input": ""})

" Voici la traduction en français : J'adore programmer."

#### Prompts with PromptTemplate

We can see that under the hood LangChain is not appending any prefix/suffix to SystemMessage's. This is because `Anthropic` has no concept of `SystemMessage`. 

Anthropic requires all prompts to end with assistant messages. This means if the last message is not an assistant message, the suffix `Assistant:` will automatically be inserted.

If you decide instead to use a normal `PromptTemplate` (one that just works on a single string) then we need do the following:

- Prefix our prompt string with `HUMAN_PROMPT` and suffix with `AI_PROMPT`

In [8]:
from anthropic_bedrock import HUMAN_PROMPT, AI_PROMPT

prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
# Format prompt for Anthropic
anthropic_prompt = PromptTemplate.from_template(
    f"{HUMAN_PROMPT}{prompt.template}{AI_PROMPT}"
)
print(anthropic_prompt)

# Invoke the Chain
chain = anthropic_prompt | bedrock_model
chain.invoke({"topic": "otters"})

PromptTemplate(input_variables=['topic'], template='\n\nHuman:Tell me a joke about {topic}\n\nAssistant:')

" Here's a silly joke about otters:\n\nWhat do you call an otter with a cold? A snotter!"

## Router Chain

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

As a very simple example, let's suppose we have four templates optimized for different types of questions, and we want to choose the template based on the user input.

Reference: <https://python.langchain.com/docs/modules/chains/foundational/router>

In [9]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [10]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [11]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [12]:
llm = bedrock_model

In [13]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [14]:
# destination_chains


{
    'physics': LLMChain(
        prompt=ChatPromptTemplate(
            input_variables=['input'],
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['input'],
                        template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"
                    )
                )
            ]
        ),
        llm=Bedrock(
            client=<botocore.client.BedrockRuntime object at 0x137d28fd0>,
            model_id='anthropic.claude-v2',
            model_kwargs={
                'max_tokens_to_sample': 1000,
                'temperature': 0.0,
                'top_k': 250,
                'top_p': 0.999,
                'stop_sequences': ['\n\nHuman:']
            }
        )
    ),
    '

#### Define a default_chain for inputs that doesn't match with any of the provided template.

In [15]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

#### Define the Multi-prompt Router prompt

In [16]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [19]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [20]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [23]:
from IPython.display import Markdown

output = chain.run(
    "List last 5 presidents of USA and their term durations. Output in a Markdown table format."
)

Markdown(output)



> Entering new MultiPromptChain chain...
History: {'input': 'List last 5 presidents of USA and their term durations. Output in a Markdown table format.'}
> Finished chain.


 Here is a list of the last 5 presidents of the USA and their term durations in Markdown table format:

| President | Term |
|-|-|  
| Donald Trump | 2017 - 2021 |
| Barack Obama | 2009 - 2017 |
| George W. Bush | 2001 - 2009 | 
| Bill Clinton | 1993 - 2001 |
| George H.W. Bush | 1989 - 1993 |

## TODO: Router Chains using LCEL